In [1]:
%load_ext autoreload
%autoreload 2

In [18]:
import torch
import torch.nn as nn
import cv2
from glasses.nn.models import *
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
from glasses.utils.Storage import ForwardModuleStorage, BackwardModuleStorage
from PIL import Image
from io import BytesIO
import requests
from torchvision.transforms import Normalize
import timm

r = requests.get('https://i.insider.com/5df126b679d7570ad2044f3e?width=700&format=jpeg&auto=webp')
im = Image.open(BytesIO(r.content))

In [34]:
from glasses.nn.models.classification.resnet import *
from glasses.nn.att import WithAtt, ECAModule, SpatialSE
from functools import partial


WithAtt(ResNetBottleneckBlock, att=SpatialSE)(32, 64)

# model = ResNet.resnet26(stem=ResNetStemC, block=WithAtt(partial(ResNetBottleneckBlock, shortcut=ResNetShorcutD), se=ECAModule))

ResNetBottleneckBlock(
  (block): Sequential(
    (conv1): Conv2dPad(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (conv2): Conv2dPad(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act2): ReLU(inplace=True)
    (conv3): Conv2dPad(16, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (se): SpatialSE(
      (avg_pool): AdaptiveAvgPool2d(output_size=1)
      (att): Sequential(
        (fc1): Linear(in_features=64, out_features=8, bias=False)
        (act1): ReLU(inplace=True)
        (fc2): Linear(in_features=8, out_features=64, bias=False)
        (act2): Sigmoid()
      )
    )
  )
  (shortcut): ResNetShorcut(
    (conv): Conv2dPad(32, 64, ke

In [4]:
model.encoder.stem[0].conv

Conv2dPad(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)

In [5]:
from transfer_weights import clone_model
from benchmark import benchmark

# tmp = ResNet.resnet50d()
# tmp = clone_model(timm.create_model('resnet50d', pretrained=True), tmp)

tmp = timm.create_model('ecaresnet50d', pretrained=True)
benchmark(tmp.eval().cuda(), ResNet.configs['resnet50d'].transform, 128)


                     top1     top5        time
efficientnet_b3   0.82040  0.96044  233.534697
cse_resnet50      0.80236  0.95070  103.796275
efficientnet_b2   0.80110  0.95118  143.738871
resnext101_32x8d  0.79312  0.94526  332.004827
wide_resnet101_2  0.78848  0.94284  234.597026
wide_resnet50_2   0.78468  0.94086  146.662350
efficientnet_b1   0.78338  0.94078  109.462508
resnet152         0.78312  0.94046  207.621832
resnext50_32x4d   0.77618  0.93698  135.171660
resnet101         0.77374  0.93546  151.992030
efficientnet_b0   0.77364  0.93560   74.319493
densenet161       0.77138  0.93560  201.173060
densenet201       0.76896  0.93370  143.988424
resnet50          0.76130  0.92862   92.407977
densenet169       0.75600  0.92806  115.986181
resnet26          0.75292  0.92570   65.222554
resnet34          0.75112  0.92288   61.915614
densenet121       0.74434  0.91972   95.509909
vgg19_bn          0.74218  0.91842  172.342654
vgg16_bn          0.73360  0.91516  152.662221
vgg19       

/home/zuppif/anaconda3/envs/dl/lib/python3.8/site-packages/torch/cuda/memory.py:231: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


(0.80614, 0.95314, 106.34580159187317)

In [6]:
from timm.models.resnet import resnet26d

timm.create_model('resnet26d')

ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Co

In [7]:
ResNet.resnet50().summary()

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Conv2dPad-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
         Conv2dPad-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
         Conv2dPad-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
        Conv2dPad-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
        Conv2dPad-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

(tensor(25557032), tensor(25557032), tensor(97.4923), tensor(439.0076))

In [8]:
class NormalizeInverse(Normalize):
    """
    Undoes the normalization and returns the reconstructed images in the input domain.
    """

    def __init__(self, mean, std):
        mean = torch.Tensor(mean)
        std = torch.Tensor(std)
        std_inv = 1 / (std + 1e-7)
        mean_inv = -mean * std_inv
        super().__init__(mean=mean_inv, std=std_inv)

    def __call__(self, tensor):
        return super().__call__(tensor.clone())

In [9]:
from glasses.interpretability import GradCam, SaliencyMap
cfg = ResNet.configs['resnet18']

transform = cfg.transform
postprocessing = Normalize(-cfg.mean / cfg.std, (1.0 / cfg.std))

x = transform(im).unsqueeze(0)
model = ResNet.resnet18(pretrained=True)

_ = model.interpret(x, using=SaliencyMap()).show()

INFO:root:Loaded resnet18 pretrained weights.


RuntimeError: Error(s) in loading state_dict for ResNet:
	Missing key(s) in state_dict: "encoder.stem.0.conv.weight", "encoder.stem.0.bn.weight", "encoder.stem.0.bn.bias", "encoder.stem.0.bn.running_mean", "encoder.stem.0.bn.running_var". 
	Unexpected key(s) in state_dict: "encoder.gate.conv.weight", "encoder.gate.bn.weight", "encoder.gate.bn.bias", "encoder.gate.bn.running_mean", "encoder.gate.bn.running_var", "encoder.gate.bn.num_batches_tracked". 

In [ ]:
def tensor2cam(image, cam):
    image_with_heatmap = image2cam(image.permute(1, 2, 0).cpu().numpy(),
                                   cam.detach().cpu().numpy())

    return torch.from_numpy(image_with_heatmap)


def image2cam(image, cam):
    h, w, c = image.shape

    cam -= np.min(cam)
    cam /= np.max(cam)  # Normalize between 0-1
    cam = cv2.resize(cam, (h, w))
    cam = np.uint8(cam * 255.0)

    img_with_cam = cv2.applyColorMap(cam, cv2.COLORMAP_JET)
    img_with_cam = cv2.cvtColor(img_with_cam, cv2.COLOR_BGR2RGB)
    img_with_cam = img_with_cam + (image * 255)
    img_with_cam /= np.max(img_with_cam)

    return img_with_cam

In [ ]:
model = ResNet.resnet18(pretrained=True).eval()
conv = model.encoder.blocks[-1].block[-1].block.conv2

weights_storage = ForwardModuleStorage(model, [conv])
gradients_storage = BackwardModuleStorage([conv])

cfg = ResNet.configs['resnet18']

transform = cfg.transform

im_tr = transform(im).unsqueeze(0)

x = Variable(im_tr, requires_grad=True)

out = model(x)

cls = torch.argmax(torch.softmax(out, dim=1))

target = torch.zeros(out.size())
target[0][int(cls)] = 1
        
out.backward(gradient=target, retain_graph=True)

In [ ]:
from torchvision.transforms import Normalize

class NormalizeInverse(Normalize):
    """
    Undoes the normalization and returns the reconstructed images in the input domain.
    """

    def __init__(self, mean, std):
        mean = torch.Tensor(mean)
        std = torch.Tensor(std)
        std_inv = 1 / (std + 1e-7)
        mean_inv = -mean * std_inv
        super().__init__(mean=mean_inv, std=std_inv)

    def __call__(self, tensor):
        return super().__call__(tensor.clone())


In [ ]:
weights = weights_storage[conv][0]
grads = gradients_storage[conv][0][0]

post_processing = NormalizeInverse(cfg.mean, cfg.std)

avg_channel_grad = F.adaptive_avg_pool2d(grads.data, 1)
cam = F.relu(torch.sum(weights * avg_channel_grad, dim=1)).squeeze(0)
image_with_heatmap = tensor2cam(post_processing(im_tr).squeeze(0), cam)


In [ ]:
import matplotlib.pyplot as plt


fig = plt.figure()

plt.imshow(image_with_heatmap)

## Get weights

Proposed API

```python
ResNet.resnet50(pretrained=True, dataset='imagenet')

ResNet.pretrained_models
```



In [ ]:
from glasses.utils.PretrainedWeightsProvider import PretrainedWeightsProvider
from pathlib import Path

provider = PretrainedWeightsProvider()
for key in provider.weights_zoo.keys():
    provider[key]

In [ ]:
from glasses.nn.models import *


# model = ResNetXt.resnext101_32x8d(pretrained=True)

VGG.vgg13(pretrained=True).summary()

In [ ]:
import torch
from glasses.nn.models import *
x = torch.rand((1,3,224,224))


FishNet.fishnet99()(x)

In [ ]:
import torch
from glasses.nn.models import *
x = torch.rand((1,3,224,224))


FishNet.fishnet99()(x)

In [ ]:
torch.load('/home/zuppif/Desktop/test.pth')

In [ ]:
from glasses.nn.blocks import Conv2dPad

x = torch.rand((1,32,9,9))

Conv2dPad(32, 64, kernel_size=9, mode = 'auto', padding=0)

In [ ]:
from glasses.nn.models import *

model =  ResNetXt.resnet18(pretrained=True)

In [ ]:
model.encoder.blocks[-1]

In [ ]:
import timm

timm.list_models('vgg')

In [ ]:
import timm
m = timm.create_model('seresnet101', pretrained=True)
m

In [ ]:
from dataclasses import dataclass
from typing import Tuple, List
from pprint import pprint
from benchmark import benchmark
from glasses.utils.ModuleTransfer import ModuleTransfer
from functools import partial
from glasses.nn.models import *
from torchvision.models import resnet18, resnet34, resnet50, resnet101, resnet152, resnext50_32x4d, resnext101_32x8d, wide_resnet50_2, wide_resnet101_2
from tqdm.autonotebook import tqdm
import torchvision.transforms as T
import torch.nn as nn
import torch
from PIL import Image
from glasses.utils.PretrainedWeightsProvider import Config


models_weights = {
    'seresnet50': Config(),
#     'efficientnet_b4': Config(resize=380, input_size=380, interpolation='bicubic')
}

zoo_models_mapping = {
    'seresnet50': [partial(timm.create_model, 'seresnet101',pretrained=True), SEResNet.cse_resnet34],
   
#   'efficientnet_b4': [partial(timm.create_model, 'tf_efficientnet_b4', pretrained=True), EfficientNet.efficientnet_b4]
}



# pprint(models_weights)

In [ ]:
model

In [ ]:
model = EfficientNetPytorch.from_pretrained('efficientnet-b0')
model.set_swish(False)
summary(model.cuda(), (3, 224, 224))